## Load Base Model

In [ ]:
from timesfm import TimesFm, freq_map, data_loader
from adapter.utils import load_adapter_checkpoint
from tqdm import tqdm
import numpy as np
import pandas as pd


tfm = TimesFm(
    context_len=512,
    horizon_len=128,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="cpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

In [ ]:
DATA_DICT = {
    "ettm2": {
        "boundaries": [34560, 46080, 57600],
        "data_path": "../datasets/ETT-small/ETTm2.csv",
        "freq": "15min",
    },
    "ettm1": {
        "boundaries": [34560, 46080, 57600],
        "data_path": "../datasets/ETT-small/ETTm1.csv",
        "freq": "15min",
    },
    "etth2": {
        "boundaries": [8640, 11520, 14400],
        "data_path": "../datasets/ETT-small/ETTh2.csv",
        "freq": "H",
    },
    "etth1": {
        "boundaries": [8640, 11520, 14400],
        "data_path": "../datasets/ETT-small/ETTh1.csv",
        "freq": "H",
    },
    "elec": {
        "boundaries": [18413, 21044, 26304],
        "data_path": "../datasets/electricity/electricity.csv",
        "freq": "H",
    },
    "traffic": {
        "boundaries": [12280, 14036, 17544],
        "data_path": "../datasets/traffic/traffic.csv",
        "freq": "H",
    },
    "weather": {
        "boundaries": [36887, 42157, 52696],
        "data_path": "../datasets/weather/weather.csv",
        "freq": "10min",
    },
}

## Load Adapter Checkpoint

Specify the adapter checkpoint path, rank and the modules used to train the adapters and whether dora was employed or not.

In [ ]:
load_adapter_checkpoint(
    model=tfm,
    adapter_checkpoint_path="./checkpoints/run_20240716_163900_lyo4psz3",
    lora_rank=1,
    lora_target_modules="all",
    use_dora=True,
)

## Test Performance

In [ ]:
dataset = "ettm1"
data_path = DATA_DICT[dataset]["data_path"]
freq = DATA_DICT[dataset]["freq"]
int_freq = freq_map(freq)
boundaries = DATA_DICT[dataset]["boundaries"]

data_df = pd.read_csv(open(data_path, "r"))

ts_cols = [col for col in data_df.columns if col != "date"]
num_cov_cols = None
cat_cov_cols = None

context_len = 512
pred_len = 96

num_ts = len(ts_cols)
batch_size = 16

dtl = data_loader.TimeSeriesdata(
    data_path=data_path,
    datetime_col="date",
    num_cov_cols=num_cov_cols,
    cat_cov_cols=cat_cov_cols,
    ts_cols=np.array(ts_cols),
    train_range=[0, boundaries[0]],
    val_range=[boundaries[0], boundaries[1]],
    test_range=[boundaries[1], boundaries[2]],
    hist_len=context_len,
    pred_len=pred_len,
    batch_size=num_ts,
    freq="15min",
    normalize=True,
    epoch_len=None,
    holiday=False,
    permute=True,
)

In [ ]:
test_batches = dtl.tf_dataset(mode="test", shift=pred_len)

In [ ]:
mae_losses = []
for batch in tqdm(test_batches.as_numpy_iterator()):
    past = batch[0]
    actuals = batch[3]
    _, forecasts = tfm.forecast(list(past), [0] * past.shape[0])
    forecasts = forecasts[:, 0 : actuals.shape[1], 5]
    mae_losses.append(np.abs(forecasts - actuals).mean())

print(f"MAE: {np.mean(mae_losses)}")